In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
store_100_path = r"df_all_store_100.parquet"
store_42_path = r"df_all_store_42.parquet"
store_4906_path = r"df_all_store_4906.parquet"

In [ ]:
df_100 = pd.read_parquet(store_100_path)
df_42 = pd.read_parquet(store_42_path)
df_4906 = pd.read_parquet(store_4906_path)
df_all = pd.concat([df_100, df_42, df_4906])

In [ ]:
del df_100, df_42, df_4906

In [ ]:
df_all['date'] = pd.to_datetime(df_all['date'])

In [ ]:
warehouse_path = r"df_all_store_VZ01.parquet"
df_warehouse = pd.read_parquet(warehouse_path)

In [ ]:
df_warehouse = df_warehouse.rename(columns={"stock": "warehouse_stock"})
df_warehouse['date'] = pd.to_datetime(df_warehouse['date'])

In [ ]:
df_warehouse['sku'] = df_warehouse['sku'].astype(str)


In [ ]:
df_warehouse = df_warehouse[df_warehouse['date'] > '2018-01-01']

In [ ]:
df_all = df_all.merge(df_warehouse[["sku","date","warehouse_stock"]], on = ["sku","date"], how = "left")


In [ ]:
df_all = df_all[df_all['date'] > '2018-01-01']

In [ ]:
df_all = df_all.sort_values('date')

In [ ]:
df_all = df_all.merge(df_all[df_all['stock'] > 0].groupby(['sku', 'store']).agg({'stock': 'first'}).reset_index().rename(columns={'stock':'palmers_first_allocation'}), on = ['sku', 'store'], how = 'left')

In [ ]:
sku_store_df_step_g = df_all.groupby(['sku', 'store'])

In [ ]:
i = 0
for store in ['42', '100']:
    for sku in ['100009161000001']:
        sku_store_df_step = sku_store_df_step_g.get_group((sku, store))
        first_stock = sku_store_df_step[sku_store_df_step["stock"]>0]["stock"].iloc[0]
        # nan in our_stock if before first stock
        sku_store_df_step.loc[sku_store_df_step["date"]<sku_store_df_step[sku_store_df_step["stock"]>0]["date"].iloc[0], "palmers_first_allocation"] = np.nan
        sku_store_df_step['days_from_first_stock'] = 1
        sku_store_df_step.loc[sku_store_df_step["date"]<sku_store_df_step[sku_store_df_step["stock"]>0]["date"].iloc[0], "days_from_first_stock"] = np.nan
        sku_store_df_step['days_from_first_stock'] = sku_store_df_step['days_from_first_stock'].cumsum()
        sku_store_df_step['sales_cumsum'] = sku_store_df_step['sales'].cumsum()
        sku_store_df_step['dynamic_stock_with_palmers_init'] = sku_store_df_step['palmers_first_allocation'] - sku_store_df_step['sales_cumsum']
        sku_store_df_step['sales_cumsum_divide_days_from_first_stock'] =  sku_store_df_step['sales_cumsum'] / sku_store_df_step['days_from_first_stock']
        zero_sales_days_back = 1
        sku_store_df_step['days_from_last_sales_that_not_zero'] = 1

        for index, row in sku_store_df_step.shift(1).iterrows():
            # if index is sku_store_df_step[sku_store_df_step["stock"]>0]["stock"] index
            if index > sku_store_df_step[sku_store_df_step["stock"]>0].index[0]:
                if row['sales'] == 0:
                    zero_sales_days_back += 1
                else:
                    zero_sales_days_back = 1

                sku_store_df_step.at[index, 'days_from_last_sales_that_not_zero'] = zero_sales_days_back
        sku_store_df_step.loc[sku_store_df_step["date"]<sku_store_df_step[sku_store_df_step["stock"]>0]["date"].iloc[0], "days_from_last_sales_that_not_zero"] = np.nan
        sku_store_df_step['rate_sales'] = sku_store_df_step['sales'] / sku_store_df_step['days_from_last_sales_that_not_zero']

In [ ]:
print(sku_store_df_step.iloc[0:30])

In [ ]:
sku_store_df_step.loc[sku_store_df_step['date'] < sku_store_df_step[sku_store_df_step["stock"]>0]['date'].iloc[0], "sum_of_our_allocation_stock"] = np.nan

In [ ]:
sku_store_df_step.loc[sku_store_df_step["date"]<sku_store_df_step[sku_store_df_step["stock"]>0]]

In [ ]:
print(df_all[(df_all['sku'] == '100009161000001') & (df_all['store'] == '100')])